# 라이브러리 import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tqdm import tqdm
from konlpy.tag import Okt
import json
import re
from matplotlib.pylab import rcParams
# rcParams['font.family'] ='malgun Gothic' #window 맑은고딕
rcParams['font.family'] ='AppleGothic' #맥os 애플고딕
rcParams['axes.unicode_minus'] = False

# 감성점수 계산

In [ ]:
# 1. 감성 사전 로드
def load_sentiword_dict(filepath='SentiWord_info.json'):
    with open(filepath, encoding='utf-8-sig') as f:
        return json.load(f)

senti_dict = load_sentiword_dict()

# 2. 단어 감성 점수 평가
def score_word(token, senti_dict):
    for entry in senti_dict:
        if entry['word'] == token:
            return int(entry['polarity'])
    return 0

# 3. 문장 감성 점수 계산
def score_sentence(sentence, senti_dict):
    okt = Okt()
    tokens = okt.pos(sentence, stem=True)

    token_list = [token[0] for token in tokens]
    scores = [score_word(token[0], senti_dict) for token in tokens]

    prenegative = ["안", "못"]
    sunegative = ["않다", "모르다"]
    adjusted_scores = []

    for idx, score in enumerate(scores):
        try:
            if token_list[idx - 1] in prenegative:
                adjusted_scores.append(score * -1)
            else:
                adjusted_scores.append(score)
        except IndexError:
            adjusted_scores.append(score)

    for idx, score in enumerate(adjusted_scores):
        try:
            if token_list[idx + 1] in sunegative:
                adjusted_scores[idx] *= -1
        except IndexError:
            pass

    adjusted_scores = [s for s in adjusted_scores if s != "None"]

    try:
        return sum(adjusted_scores) / len(adjusted_scores)
    except ZeroDivisionError:
        return 0
    
# 4. 데이터프레임 감성 점수 계산 (브랜드명 포함 여부 확인)
def calculate_sentiment_by_competitor(df, competitor_dict, senti_dict):
    """
    경쟁사 이름이 포함된 문장을 필터링하여 감성 점수를 계산
    """
    results = []

    for competitor, variations in competitor_dict.items():
        # 경쟁사 이름이 포함된 문장 필터링 (여러 표현을 탐색)
        competitor_sentences = df[df["sentence"].apply(lambda x: any(v in x for v in variations))]
        
        # 감성 점수 계산
        tqdm.pandas(desc=f"Calculating Sentiment for {competitor}")
        competitor_sentences["sentiment_score"] = competitor_sentences["sentence"].progress_apply(
            lambda x: score_sentence(x, senti_dict)
        )
        
        # 결과 저장
        results.append({
            "competitor": competitor,
            "review_count": len(competitor_sentences),
            "avg_sentiment_score": competitor_sentences["sentiment_score"].mean() if len(competitor_sentences) > 0 else 0
        })
    
    return pd.DataFrame(results)

def generate_positioning_map_data_with_minmax_scaling(df):
    """
    리뷰 개수와 평균 감성 점수를 Min-Max Scaling으로 정규화
    """
    # Min-Max Scaling
    scaler = MinMaxScaler()
    df[['scaled_review_count', 'scaled_sentiment_score']] = scaler.fit_transform(
        df[['review_count', 'avg_sentiment_score']]
    )
    return df


## 브랜드명

In [ ]:
# 4. 경쟁사 브랜드 목록 (다양한 표현 포함)
competitor_dict = {
    "엘지": ["LG", "엘지", "lg", "엘X", "L G", "엘쥐", '로보킹'],
    "삼성": ["삼성", '비스포크'],
    "로보락": ["로보락", "큐레보", 'Qrevo', 'qrevo'],
    "드리미": ["드리미", "l ", 'x '],
    "에코백스": ["에코백스", '옴니'],
    "나르왈" : ['나르왈', '프레오']
}

## Positioning Map

In [ ]:
def plot_positioning_map_with_positive_line(result, weight_review=0.5, weight_sentiment=0.5):
    x_values = result['scaled_review_count']
    y_values = result['scaled_sentiment_score']
    competitors = result['competitor']

    # 경쟁사별 색상 지정
    colors = {
        "엘지": "firebrick",
        "로보락": "red",
        "삼성": "blue",
        "드리미": "brown",
        "나르왈": "purple",
        "에코백스": "navy"
    }

    plt.figure(figsize=(10, 7))

    # 감성 점수 기반 Mobility Barrier 계산
    z = np.polyfit(x_values, y_values, 1)
    p = np.poly1d(z)

    # 점 그리기
    for i, competitor in enumerate(competitors):
        plt.scatter(
            x_values[i],
            y_values[i],
            color=colors.get(competitor, "gray"),
            s=200,
            alpha=0.8,
            edgecolors="black",
            marker='o',
            label=competitor if competitor not in plt.gca().get_legend_handles_labels()[1] else ""
        )
        plt.text(
            x_values[i] + 0.02,
            y_values[i],
            competitor,
            fontsize=20,
            color=colors.get(competitor, "black")
        )

    # 추세선 추가
    plt.plot(x_values, p(x_values), "gray", linestyle="--", label="추세선")

    # 그래프 꾸미기
    plt.xlabel("interest", fontsize=12)
    plt.ylabel("Sentiment", fontsize=12)
    plt.title("Positioning Map", fontsize=14)
    #plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# 최종 실행

### 실행

In [ ]:
# 8. 실행 흐름
if __name__ == "__main__":
    # 리뷰 데이터 로드
    file_path = '[DIC]positioning_map.csv'  # 로봇청소기 관련 크롤링 데이터
    df = pd.read_csv(file_path, encoding = 'utf-8-sig')
    df = df[df['topic'] == 5]

    # 경쟁사별 감성 점수 계산 (브랜드명이 포함된 문장만 분석)
    competitor_data = calculate_sentiment_by_competitor(df, competitor_dict, senti_dict)

    # 포지셔닝 맵 데이터 생성 (스케일링 포함)
    positioning_data = generate_positioning_map_data_with_minmax_scaling(competitor_data)

    # 포지셔닝 맵 시각화
    plot_positioning_map_with_positive_line(positioning_data)